In [1]:
import findspark
findspark.init('/usr/local/spark-3.1.2-bin-hadoop3.2')

import pymongo
import pymongo_spark
pymongo_spark.activate()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, to_date

In [2]:
# spark.mongodb.input.uri=mongodb://127.0.0.1/databaseName.collectionName
## PySpark connector model
# spark = SparkSession.builder.appName("Spark Benchmarking") \
#                 .config("spark.mongodb.input.uri", mongodb_uri)\
#                 .config("spark.mongodb.output.uri", mongodb_uri)\
#                 .config("spark.jars.packages",
#                         "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1",)\
#                 .getOrCreate()

In [3]:
mongodb_uri = "mongodb://mongo_db:27017/covid-project.covid_deaths"
spark = SparkSession.builder.appName("Spark Benchmarking") \
                .config("spark.mongodb.output.uri", mongodb_uri)\
                .config("spark.jars.packages",
                        "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1",)\
                .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-793fe352-ec09-4339-b416-c13be9f3b645;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 350ms :: artifacts dl 9ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       c

In [4]:
covid_deaths = spark.read.parquet('../data/covid_deaths.parquet')
covid_deaths.write.format('mongo').mode('append')\
                                    .option('database', 'covid-project')\
                                    .option('collection', 'covid_deaths')\
                                    .save()

21/07/11 16:58:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
covid_vaccinations = spark.read.parquet('../data/covid_vaccinations.parquet')
covid_vaccinations.write.format('mongo').mode('append')\
                                        .option('database', 'covid-project')\
                                        .option('collection', 'covid_vaccinations')\
                                        .save()